# Internal Notebook for `TLMResponses` Modification

To allow TLMResponses to parse responses that use web search or file search, we modify the `_format_tools_prompt` function to convert OpenAI Responses built-in tools into function tools, just like the ones users can insert. This way, the agent has context for when the agent uses these tools. These tools are formatted into the system message the same way other user-created functions have been created. The code for this is shown below:

```py
elif tool["type"] == "file_search":
    tool_dict = {
        "type": "function",
        "name": "file_search",
        "description": "Search user-uploaded documents for relevant passages.",
        "parameters": {
            "type": "object",
            "properties": {
                "queries": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Search queries to run against the document index.",
                },
            },
            "required": ["queries"],
        },
    }
elif tool["type"] == "web_search_preview":
    tool_dict = {
        "type": "function",
        "name": "web_search_call",
        "description": "Search the web for relevant information.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search the web with a query and return relevant pages.",
                },
            },
            "required": ["query"],
        },
    }
```

Now that the TLM has an understanding of the OpenAI tools it has available to it, we simply have to handle the conversion of the tool calls in the response into expected function calls and responses that are ready for TLM scoring.

## File Search

For file search, we first need to check if the message can be used for TLM scoring using fetched content. We can do this by checking `message["results"]` which is populated when the user correctly has `include=['file_search_call.results']` in the OpenAI Responses request and unpopulated otherwise. If it's empty, we send out a warning and skip TLM scoring for the specific file search call (the rest of the message chain gets scored still using the normal process). If it has content, however, we can begin processing.

```py
elif message["type"] == "file_search_call":
    if message["results"] == None:
        warnings.warn(
            f"File search call returned no results. Please include include=['file_search_call.results'] in your request.",
            UserWarning,
            stacklevel=2,
        )
        continue
```

First, we set up a tool call object so the TLM can see that the agent tries to make a call to the file search tool. This will look like a user-generated tool call, but in reality, it is an OpenAI-specific one.

```py
tool_call = {
    "name": "file_search",
    "arguments": {"queries": message["queries"]},
    "call_id": message["id"],
}

if i == 0 or _get_role(messages[i - 1]) != _ASSISTANT_ROLE:
    content_parts.append(_ASSISTANT_PREFIX)
content_parts.append(
    f"{_TOOL_CALL_TAG_START}\n{json.dumps(tool_call, indent=2)}\n{_TOOL_CALL_TAG_END}"
)
```

We can then show the TLM a tool call response object so that it can see the result of the file search tool call.

```py
results_list = [
    {
        "attributes": result["attributes"],
        "file_id": result["file_id"],
        "filename": result["filename"],
        "score": result["score"],
        "text": result["text"],
    }
    for result in message["results"]
]

tool_call_response = {
    "name": "file_search",
    "call_id": message["id"],
    "output": results_list,
}

content_parts.append("")
content_parts.append(_TOOL_PREFIX)
content_parts.append(
    f"{_TOOL_RESPONSE_TAG_START}\n{json.dumps(tool_call_response, indent=2)}\n{_TOOL_RESPONSE_TAG_END}"
)
```

The next message that gets processed is the assistant message, so it will function as normal, now with the additional context.

## Web Search

Web search works in a very similar way to file search, only with a different means of fetching data. Because of this similarity, I will only show the real differences between the two.

To fetch data, we first need a list of the URLs to search. This can be done by extracting URLs from the assistant message's annotations.

```py
output_message = [
    m["content"][0] for m in messages[i + 1 :] if m["type"] == "message"
][0]

if output_message["type"] == "refusal":
    continue

annotations = list(
    set(
        [
            (annotation["url"], annotation["title"])
            for annotation in output_message["annotations"]
            if annotation["type"] == "url_citation"
        ]
    )
)
```

We can then use `trafilatura` to extract the main content from the fetched web pages. We also cache these entries because website data may not be stored by the user in message history and can be expensive to re-fetch. The requests for fetching different URLs are done in parallel.

```py
with ThreadPoolExecutor() as executor:
    def extract_text(pair):
        url = pair[0]
        if url in _url_cache:
            return _url_cache[url]
        response = extract(fetch_url(url), output_format="markdown")
        _url_cache[url] = response
        return response

    requests = list(
        executor.map(
            extract_text,
            annotations,
        )
    )
```

We can then put this into a tool call response like before:

```py
websites = [
    {
        "url": url,
        "title": title,
        "content": data,
    }
    for (url, title), data in zip(annotations, requests)
]

content_parts.append(_TOOL_PREFIX)
content_parts.append(
    f"{_TOOL_RESPONSE_TAG_START}\n{json.dumps(tool_response, indent=2)}\n{_TOOL_RESPONSE_TAG_END}"
)
```

## Testing

Both of these methods have been tested and have proper responses. Additionally, the code for handling message has been refactored to use the same handling function for prompt and response. This is to make future modifications easier to implement. However, this has had the side-effect of making messages go on another line from the role.

Eg.
```
User:
Message
```
Instead of the previous
```
User: Message
```

I don't think this is a major issue, makes message formatting more consistent, and reduces complexity without any real impact on accuracy, so I think this change should be kept. However, some tests in `utils/chat.py` may need to be modified.